In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-20 13:04:38.319209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 13:04:38.886119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-20 13:04:39.552633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 13:04:39.576809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
subject = 'Amapá - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - Consumo de Cimento (t)
0,2003-1,0.711421,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.035813e+06,356591.003430,10.883143,7.033593e+06,8.514392,10.392
1,2003-2,0.711553,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.046432e+06,356647.711004,10.885206,7.037356e+06,8.508753,6.857
2,2003-3,0.711685,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.057052e+06,356704.418579,10.887268,7.041120e+06,8.503114,7.011
3,2003-4,0.711817,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.067671e+06,356761.126153,10.889331,7.044883e+06,8.497475,7.122
4,2003-5,0.711949,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.078290e+06,356817.833728,10.891394,7.048646e+06,8.491835,5.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.208
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.476
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.236
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.549


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amapá - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.388995,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.179199,-1.457028,-1.470096,1.047494,1.300799
158,1.332036,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.184540,-1.486109,-1.464204,1.056372,1.292722
159,1.275077,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.189880,-1.515190,-1.458311,1.065250,1.284645
160,1.218118,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.195221,-1.544271,-1.452419,1.074128,1.276568


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
157     8.322
158    11.373
159     9.651
160    10.855
161    13.970
Name: Amapá - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [11]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 13)

In [12]:
# input para treinamento
test_input = input_data.iloc[train_split-window_size:split_index + 1]
test_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
126,0.639896,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.813802,0.876851,-0.305631,0.923476,0.978734
127,0.580720,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.829759,0.846024,-0.368234,0.925918,0.993895
128,0.521544,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.845716,0.815197,-0.430837,0.928361,1.009056
129,0.462368,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.861673,0.784370,-0.493440,0.930803,1.024217
130,0.403191,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.877629,0.753543,-0.556042,0.933246,1.039378
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [13]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test_target

array([12.399, 11.268,  8.741, 12.358, 11.876, 11.713, 11.449,  7.805,
        7.726,  8.516,  8.503, 10.175, 11.775,  9.724, 10.069, 13.93 ,
       11.918, 11.757,  9.691,  7.378,  6.97 ,  7.612,  8.442,  8.835,
       11.251, 11.342, 13.11 , 13.195, 10.053, 12.274])

In [14]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [15]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1163406737, 960475672, 4216526090, 3019472274, 2975135380, 3983919895, 2220739625, 2789345229, 700955648, 4124268824, 996484512, 1676813750, 662915029, 3074102579, 2384568541, 855032732, 2992321060, 2436767679, 1535099939, 921828810]


Step: 0 ___________________________________________


2023-10-20 13:04:39.932395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 13:04:39.932603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 13:04:39.932747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 5.775665760040283
winner_seed: 1163406737


Step: 1 ___________________________________________
val_loss: 5.720895767211914
winner_seed: 960475672


Step: 2 ___________________________________________
val_loss: 5.622644901275635
winner_seed: 4216526090


Step: 3 ___________________________________________
val_loss: 5.5852580070495605
winner_seed: 3019472274


Step: 4 ___________________________________________
val_loss: 5.36119270324707
winner_seed: 2975135380


Step: 5 ___________________________________________
val_loss: 14.806296348571777


Step: 6 ___________________________________________
val_loss: 5.801276683807373


Step: 7 ___________________________________________
val_loss: 6.961278438568115


Step: 8 ___________________________________________
val_loss: 8.389022827148438


Step: 9 ___________________________________________
val_loss: 9.070533752441406


Step: 10 ___________________________________________
val_loss: 6.893496513366699


Step: 11 ____________________

2023-10-20 13:16:38.700482: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 6.286588668823242


Step: 14 ___________________________________________


2023-10-20 13:17:50.094022: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 5.526735782623291


Step: 15 ___________________________________________
val_loss: 5.005926132202148
winner_seed: 855032732


Step: 16 ___________________________________________
val_loss: 14.902663230895996


Step: 17 ___________________________________________
val_loss: 6.7793073654174805


Step: 18 ___________________________________________
val_loss: 7.607578754425049


Step: 19 ___________________________________________
val_loss: 12.04067611694336


final_seed: 855032732


In [18]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 131ms/step - loss: 239.6277 - val_loss: 104.0686
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 164.4232 - val_loss: 64.2777
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 132.4984 - val_loss: 53.9576
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 120.4036 - val_loss: 47.6530
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 111.7325 - val_loss: 42.2826
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 104.7266 - val_loss: 37.4515
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 97.8151 - val_loss: 33.1030
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 91.5791 - val_loss: 29.1488
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 85.6659 - val_loss: 25.5781
Epoch 10/10000
4/4 [==============================] - 0s 11ms/step - loss:

Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7869 - val_loss: 10.8715
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7802 - val_loss: 10.8434
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7855 - val_loss: 10.8620
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7767 - val_loss: 10.8283
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7785 - val_loss: 10.7568
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7905 - val_loss: 10.6883
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7794 - val_loss: 10.7294
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7796 - val_loss: 10.7366
Epoch 89/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7838 - val_loss: 10.7324
Epoch 90/10000
4/4 [==============================] - 0s 11ms/step - loss

Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7799 - val_loss: 11.0292
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7908 - val_loss: 10.9842
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7836 - val_loss: 10.9768
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7791 - val_loss: 11.0144
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7812 - val_loss: 11.0386
Epoch 165/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7836 - val_loss: 11.0513
Epoch 166/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7845 - val_loss: 11.0747
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7855 - val_loss: 11.0195
Epoch 168/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7801 - val_loss: 10.9922
Epoch 169/10000
4/4 [==============================] - 0s 11ms/s

Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7976 - val_loss: 10.7099
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 31.8048 - val_loss: 10.6287
Epoch 241/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.7767 - val_loss: 10.7476
Epoch 242/10000
4/4 [==============================] - 0s 13ms/step - loss: 31.7680 - val_loss: 10.9259
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7891 - val_loss: 11.1229
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7853 - val_loss: 11.2248
Epoch 245/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7959 - val_loss: 11.3705
Epoch 246/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7980 - val_loss: 11.3966
Epoch 247/10000
4/4 [==============================] - 0s 9ms/step - loss: 31.7931 - val_loss: 11.3261
Epoch 248/10000
4/4 [==============================] - 0s 10ms/step -

4/4 [==============================] - 0s 11ms/step - loss: 15.5364 - val_loss: 30.1282
Epoch 319/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.4578 - val_loss: 31.1189
Epoch 320/10000
4/4 [==============================] - 0s 11ms/step - loss: 13.4867 - val_loss: 32.0622
Epoch 321/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.8710 - val_loss: 33.5948
Epoch 322/10000
4/4 [==============================] - 0s 11ms/step - loss: 12.6385 - val_loss: 35.2910
Epoch 323/10000
4/4 [==============================] - 0s 12ms/step - loss: 12.6393 - val_loss: 36.8659
Epoch 324/10000
4/4 [==============================] - 0s 12ms/step - loss: 11.8296 - val_loss: 38.5973
Epoch 325/10000
4/4 [==============================] - 0s 11ms/step - loss: 12.3294 - val_loss: 40.0842
Epoch 326/10000
4/4 [==============================] - 0s 11ms/step - loss: 11.7054 - val_loss: 41.5078
Epoch 327/10000
4/4 [==============================] - 0s 11ms/step - loss: 11.7

...

4/4 [==============================] - 0s 11ms/step - loss: 6.4904 - val_loss: 7.6815
Epoch 1190/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.6121 - val_loss: 7.1327
Epoch 1191/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.8970 - val_loss: 6.9972
Epoch 1192/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2529 - val_loss: 6.8232
Epoch 1193/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4340 - val_loss: 6.8342
Epoch 1194/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3235 - val_loss: 6.7437
Epoch 1195/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7509 - val_loss: 7.2413
Epoch 1196/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.7399 - val_loss: 8.0232
Epoch 1197/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.9387 - val_loss: 8.8760
Epoch 1198/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3918 - val_

4/4 [==============================] - 0s 11ms/step - loss: 5.6271 - val_loss: 10.3967
Epoch 1269/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.1611 - val_loss: 11.2007
Epoch 1270/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5203 - val_loss: 10.7328
Epoch 1271/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.7174 - val_loss: 9.6608
Epoch 1272/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2773 - val_loss: 8.5001
Epoch 1273/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.2481 - val_loss: 9.2261
Epoch 1274/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.7995 - val_loss: 9.8202
Epoch 1275/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.7630 - val_loss: 10.6406
Epoch 1276/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2672 - val_loss: 11.3517
Epoch 1277/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.0281 -

Epoch 1348/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.6490 - val_loss: 7.5745
Epoch 1349/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.8983 - val_loss: 6.4978
Epoch 1350/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5342 - val_loss: 5.9272
Epoch 1351/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.7490 - val_loss: 6.1994
Epoch 1352/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4713 - val_loss: 7.0059
Epoch 1353/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2717 - val_loss: 7.2323
Epoch 1354/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.4803 - val_loss: 7.2937
Epoch 1355/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.1781 - val_loss: 7.7709
Epoch 1356/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.8054 - val_loss: 8.0292
Epoch 1357/10000
4/4 [==============================] - 0s 9ms/step - loss: 

4/4 [==============================] - 0s 12ms/step - loss: 6.8886 - val_loss: 9.4289
Epoch 1428/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7208 - val_loss: 11.0201
Epoch 1429/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.0195 - val_loss: 10.9632
Epoch 1430/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7471 - val_loss: 11.2492
Epoch 1431/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.7846 - val_loss: 8.5287
Epoch 1432/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.2014 - val_loss: 6.9776
Epoch 1433/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.6990 - val_loss: 6.8851
Epoch 1434/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.2115 - val_loss: 7.2535
Epoch 1435/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.7857 - val_loss: 7.7795
Epoch 1436/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.8318 - v

4/4 [==============================] - 0s 10ms/step - loss: 6.0178 - val_loss: 7.4283
Epoch 1507/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.0316 - val_loss: 7.5967
Epoch 1508/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.2842 - val_loss: 7.4721
Epoch 1509/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.9441 - val_loss: 7.3340
Epoch 1510/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1049 - val_loss: 8.1562
Epoch 1511/10000
4/4 [==============================] - 0s 14ms/step - loss: 5.8521 - val_loss: 9.5908
Epoch 1511: early stopping


In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,13.919381,13.990292,14.04735,14.09381,14.126345,14.146803,14.180419,14.265699,14.312064,14.341601,14.367002,14.400402,14.36585,14.369029,14.437085,14.431826,14.41492,14.354415,14.35275,14.413462,14.413156,14.375886,14.388503,14.356398,14.329171,14.334376,14.284247,14.326163,14.356879,14.575981
Target,12.399,11.268,8.741,12.358,11.876,11.713,11.449,7.805,7.726,8.516,8.503,10.175,11.775,9.724,10.069,13.93,11.918,11.757,9.691,7.378,6.97,7.612,8.442,8.835,11.251,11.342,13.11,13.195,10.053,12.274
Error,1.520381,2.722292,5.30635,1.73581,2.250344,2.433803,2.731419,6.4607,6.586064,5.825602,5.864001,4.225402,2.590851,4.645029,4.368085,0.501825,2.49692,2.597415,4.66175,7.035462,7.443156,6.763886,5.946503,5.521398,3.078171,2.992376,1.174248,1.131164,4.303879,2.301981


In [21]:
display(mae)
display(mape)

3.9072087

0.42878637

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[170.19118]] - Target[122.52900000000001]| =  Error: [[47.662178]]; MAPE:[[0.38898692]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[172.67328]] - Target[118.101]| =  Error: [[54.57228]]; MAPE:[[0.46208146]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[86.20682]] - Target[71.22500000000001]| =  Error: [[14.981819]]; MAPE:[[0.21034496]]


[array([[47.662178]], dtype=float32),
 array([[54.57228]], dtype=float32),
 array([[14.981819]], dtype=float32)]

39.072094

0.35380444